In [13]:
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire import webdriver


import gzip
import io

In [2]:
options = webdriver.ChromeOptions()

options.add_argument(r"--user-data-dir=C:\Users\karol\AppData\Local\Google\Chrome\'User Data'")

options.add_argument("--profile-directory='Profile 2'")
# options.add_argument("--user-data-dir=/mnt/c/Users/karol/AppData/Local/Google/Chrome/User Data")
# options.add_argument("--profile-directory=Profile 2")

options.add_argument('--start-maximized')

service = Service(executable_path=r"C:\Users\karol\miniconda3\envs\scraper_conda\Scripts\chromedriver.exe")

# options.add_argument("--headless")



In [3]:
driver = webdriver.Chrome(service=service, options=options)

In [4]:
driver.get("https://borg.wolterskluwer.pl")

In [5]:
if driver.find_elements(By.ID, "login"):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "login"))
    ).send_keys("xolar1989@o2.pl")

    driver.find_element(By.ID, "password").send_keys("Awglm12345!")

    driver.find_element(By.ID, "remember_btn").click()
    driver.find_element(By.ID, "login_btn").click()

In [6]:
r = driver.find_element(By.CSS_SELECTOR, '.product_item_container')


r.click()

In [7]:
original_window = driver.current_window_handle

# Wait for the new window
WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

# Switch to the new window
for handle in driver.window_handles:
    if handle != original_window:
        driver.switch_to.window(handle)
        break

In [8]:
if driver.find_elements(By.XPATH, '//*[@id="lex"]/div/header/ng-include/div/nav/div/div[1]/app-left-menu-button/div/button'):
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="lex"]/div/header/ng-include/div/nav/div/div[1]/app-left-menu-button/div/button'))
    )
    element.click()

In [9]:
driver.get("https://sip.lex.pl/#/search-resources?p=%7B%22documentMainType%22:%22QA%22,%22domains%22:%5B%7B%22label%22:%22Podatki%22,%22conceptId%22:4697%7D%5D,%22qaValidity%22:%7B%22nrs%22:134217729,%22label%22:%22aktualne%22%7D%7D&sortBy=default")


In [10]:
matching_requests = [
    request for request in driver.requests
    if request.response and "searchResults.get.json" in request.url
]


In [ ]:
rr = 4

In [14]:


for request in matching_requests:
    try:
        compressed = io.BytesIO(request.response.body)
        decompressed = gzip.GzipFile(fileobj=compressed).read()
        data = json.loads(decompressed.decode('utf-8'))
        print(json.dumps(data, indent=2))
    except Exception as e:
        print("❌ Błąd dekodowania:", e)

{
  "debug": {
    "icsRequestTime": 0,
    "dbServicesRequestTime": 0
  },
  "documentList": [
    {
      "title": "Czy brak wyodr\u0119bnienia pozycji dotycz\u0105cych kapita\u0142\u00f3w wypracowanych przed przyst\u0105pieniem do rycza\u0142tu w sprawozdaniu finansowym skutkuje pozbawieniem podatnika prawa do stosowania esto\u0144skiego CIT?",
      "owner": true,
      "shared": false,
      "resourceType": "DOCUMENT",
      "ident": "0DOCUMENT",
      "nro": 623753473,
      "versionId": 623753473,
      "lastAccessTime": 0,
      "type": "QA",
      "monographFragmentId": 0,
      "stateName": "question-and-answer",
      "hasToc": false,
      "noveltyDate": 1752105600000,
      "documentProductionType": "QA",
      "documentType": "QA",
      "answerDate": 1752098400000,
      "firstAuthorName": "Kowalski Rados\u0142aw",
      "editionPlace": "QA 2996481",
      "validity": "ACTUAL",
      "validityFrom": -5364666000000,
      "validityTo": 26223778800000,
      "newInProduct"

In [17]:
seen_edition_places = set()
rows_of_documents = []

In [20]:
# def get_
for r in driver.requests:
    if not (r.response and "searchResults.get.json" in r.url):
        continue
    try:
        compressed = io.BytesIO(r.response.body)
        decompressed = gzip.GzipFile(fileobj=compressed).read()
        data = json.loads(decompressed.decode('utf-8'))
        document_list = data.get("documentList", [])
        for doc in data.get("documentList", []):
            ep = doc.get("editionPlace")
            if ep and ep not in seen_edition_places:
                rows_of_documents.append(doc)
                seen_edition_places.add(ep)

    except Exception as e:
        print(f"❌ Błąd przy parsowaniu odpowiedzi: {e}")

In [15]:
r =4

In [ ]:
for _ in range(20):  # liczba scrolli — dostosuj do potrzeby
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7)  # poczekaj na załadowanie wyników
    new_requests = [
        r for r in driver.requests
        if r.response and "searchResults.get.json" in r.url and r.url not in seen_urls
    ]